In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pycountry as pycountry
import geopandas
%matplotlib inline

plt.style.use('default')

sns.set(style='whitegrid')

In [ ]:
df = pd.read_csv('events.csv', low_memory=False)

df = df.dropna(subset=['city'])
df = df[df.city != 'Unknown']
df = df['country'].value_counts().to_frame()
df['months'] = df.index
df = df.rename(columns={'country': 'events', 'months': 'country'})

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
    
df['country'] = df['country'].replace(countries)
df = df[df.country != 'Venezuela']
df = df[df.country != 'Russia']
df

In [ ]:
plotly.tools.set_credentials_file(username='Florencia-97', api_key='VQgR2XhfedXEFIdPr9TT')

df.head()

data = [dict(
        type = 'choropleth',
        locations = df['country'],
        z = df['events'],
        text = df['country'],
        colorscale = 'Viridis',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = {'title':'Events'},
      ) ]

layout = dict(
    title = 'Events by Country',
    geo = dict(
        showframe = True,
        showcoastlines = True,
        projection = dict(
            type = 'mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot(fig, filename='d3-world-map')


## Dado que obtuvimos que la mayor cantidad de trafico proviene de Brasil, veremos las regiones:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [ ]:
fp = "BRA_adm1.shp"
map_df = gpd.read_file(fp)
map_df.head(260)

In [ ]:

f, ax = plt.subplots(1, figsize=(15, 15))
ax = map_df.plot(ax=ax)
ax.set_axis_off()
plt.show()

#map_df.plot()

In [ ]:
df = pd.read_csv('events.csv', low_memory=False)

In [ ]:
filter = df[df["country"]=="Brazil"]

In [ ]:
filter2 = filter[filter["region"]!="Unknown"]["region"].value_counts().to_frame()
filter2.reset_index(inplace=True)

In [ ]:
filter2

In [ ]:
filter2["index"]= filter2["index"].replace(to_replace="Sao Paulo",value="São Paulo")
filter2["index"]= filter2["index"].replace(to_replace="Federal District",value="Distrito Federal")
filter2["index"]= filter2["index"].replace(to_replace="Ceara",value="Ceará")
filter2["index"]= filter2["index"].replace(to_replace="Parana",value="Paraná")
filter2["index"]= filter2["index"].replace(to_replace="Amapa",value="Amapá")
filter2["index"]= filter2["index"].replace(to_replace="Espirito Santo",value="Espírito Santo")
filter2["index"]= filter2["index"].replace(to_replace="Goias",value="Goiás")
filter2["index"]= filter2["index"].replace(to_replace="Maranhao",value="Maranhão")
filter2["index"]= filter2["index"].replace(to_replace="Para",value="Pará")
filter2["index"]= filter2["index"].replace(to_replace="Piaui",value="Piauí")
filter2["index"]= filter2["index"].replace(to_replace="Rondonia",value="Rondônia")


In [ ]:
merged = map_df.set_index('NAME_1').join(filter2.set_index('index'))
merged["region"] = merged["region"].fillna(0)

as_list = merged.index.tolist()
idx = as_list.index('Distrito Federal')
as_list[idx] = 'D.F.'
merged.index = as_list
merged.head(555)

In [ ]:
ax = merged.plot(column="region", cmap='Wistia', figsize=(16, 10),edgecolor="white",legend=True)
ax.set_title("Mapa coroplético  de las regiones de Brasil con mayor actividad",fontsize=20)
ax.text(-16, -15, 'Cantidad',
        rotation='vertical',fontsize=15)

ax.set_xlabel("Latitud",fontsize=15)
ax.set_ylabel("Latitud",fontsize=15)
merged.apply(lambda x: ax.annotate(s=x.name, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);
